In [63]:
import math as m
import numpy as np
from numpy import genfromtxt
import os
from tqdm import tqdm

num_kp = 22
num_axes = 3

# List all the files in dir
kp_path = "../dataset/Movements/Kinect/Positions/"
ka_path = "Movements/Kinect/Angles/"
kinect_positions = sorted([kp_path+f for f in os.listdir(kp_path) if f.endswith('.txt')])
kinect_angles = sorted([ka_path+f for f in os.listdir(ka_path) if f.endswith('.txt')])

In [2]:
def Rotx(theta):
    return np.matrix([[ 1, 0           , 0           ],
                      [ 0, m.cos(theta),-m.sin(theta)],
                      [ 0, m.sin(theta), m.cos(theta)]])
  
def Roty(theta):
    return np.matrix([[ m.cos(theta), 0, m.sin(theta)],
                      [ 0           , 1, 0           ],
                      [-m.sin(theta), 0, m.cos(theta)]])
  
def Rotz(theta):
    return np.matrix([[ m.cos(theta), -m.sin(theta), 0 ],
                      [ m.sin(theta), m.cos(theta) , 0 ],
                      [ 0           , 0            , 1 ]])

def eulers_2_rot_matrix(x):
    gamma_x=x[0];beta_y=x[1];alpha_z=x[2];
    return Rotz(alpha_z)*Roty(beta_y)*Rotx(gamma_x)

In [3]:
# convert the data from relative coordinates to absolute coordinates
def rel2abs(p, a, num_frames):
    skel = np.zeros((num_kp, num_axes, num_frames))
    for i in range(num_frames):
        """
        1 Waist (absolute)
        2 Spine
        3 Chest
        4 Neck
        5 Head
        6 Head tip
        7 Left collar
        8 Left upper arm 
        9 Left forearm
        10 Left hand
        11 Right collar
        12 Right upper arm 
        13 Right forearm
        14 Right hand
        15 Left upper leg 
        16 Left lower leg 
        17 Left foot 
        18 Left leg toes
        19 Right upper leg 
        20 Right lower leg 
        21 Right foot
        22 Right leg toes
        """

        joint = p[:,:,i]
        joint_ang = a[:,:,i]

        # chest, neck, head
        rot_1 = eulers_2_rot_matrix(joint_ang[0,:]*np.pi/180);
        joint[1,:] =  rot_1@joint[1,:] + joint[0,:]
        rot_2 = rot_1*eulers_2_rot_matrix(joint_ang[1,:]*np.pi/180)
        joint[2,:] =  rot_2@joint[2,:] +  joint[1,:]
        rot_3 = rot_2*eulers_2_rot_matrix(joint_ang[2,:]*np.pi/180)
        joint[3,:] =  rot_3@joint[3,:] +  joint[2,:]
        rot_4 = rot_3*eulers_2_rot_matrix(joint_ang[3,:]*np.pi/180)
        joint[4,:] =  rot_4@joint[4,:] +  joint[3,:]
        rot_5 = rot_4*eulers_2_rot_matrix(joint_ang[4,:]*np.pi/180)
        joint[5,:] =  rot_5@joint[5,:] +  joint[4,:]

        # left-arm
        rot_6 = eulers_2_rot_matrix(joint_ang[2,:]*np.pi/180)
        joint[6,:] =  rot_6@joint[6,:] +  joint[2,:]
        rot_7 = rot_6*eulers_2_rot_matrix(joint_ang[6,:]*np.pi/180)
        joint[7,:] =  rot_7@joint[7,:] +  joint[6,:]
        rot_8 = rot_7*eulers_2_rot_matrix(joint_ang[7,:]*np.pi/180)
        joint[8,:] = rot_8@joint[8,:] +  joint[7,:]
        rot_9 = rot_8*eulers_2_rot_matrix(joint_ang[8,:]*np.pi/180)
        joint[9,:] = rot_9@joint[9,:] +  joint[8,:]

        # right-arm
        rot_10 = eulers_2_rot_matrix(joint_ang[2,:]*np.pi/180)
        joint[10,:] =  rot_10@joint[10,:] +  joint[2,:]
        rot_11 = rot_10*eulers_2_rot_matrix(joint_ang[10,:]*np.pi/180)
        joint[11,:] =  rot_11@joint[11,:] +  joint[10,:]
        rot_12 = rot_11*eulers_2_rot_matrix(joint_ang[11,:]*np.pi/180)
        joint[12,:] = rot_12@joint[12,:] +  joint[11,:]
        rot_13 = rot_12*eulers_2_rot_matrix(joint_ang[12,:]*np.pi/180)
        joint[13,:] = rot_13@joint[13,:] +  joint[12,:]

        # left-leg
        rot_14 = eulers_2_rot_matrix(joint_ang[0,:]*np.pi/180)
        joint[14,:] =  rot_14@joint[14,:] +  joint[0,:]
        rot_15 = rot_14*eulers_2_rot_matrix(joint_ang[14,:]*np.pi/180)
        joint[15,:] =  rot_15@joint[15,:] +  joint[14,:]
        rot_16 = rot_15*eulers_2_rot_matrix(joint_ang[15,:]*np.pi/180)
        joint[16,:] = rot_16@joint[16,:] +  joint[15,:]
        rot_17 = rot_16*eulers_2_rot_matrix(joint_ang[16,:]*np.pi/180)
        joint[17,:] = rot_17@joint[17,:] +  joint[16,:]

        # right-leg
        rot_18 = eulers_2_rot_matrix(joint_ang[0,:]*np.pi/180)
        joint[18,:] =  rot_18@joint[18,:] +  joint[0,:]
        rot_19 = rot_18*eulers_2_rot_matrix(joint_ang[18,:]*np.pi/180)
        joint[19,:] =  rot_19@joint[19,:] +  joint[18,:]
        rot_20 = rot_19*eulers_2_rot_matrix(joint_ang[19,:]*np.pi/180)
        joint[20,:] = rot_20@joint[20,:] +  joint[19,:]
        rot_21 = rot_20*eulers_2_rot_matrix(joint_ang[20,:]*np.pi/180)
        joint[21,:] = rot_21@joint[21,:] +  joint[20,:]

        skel[:,:,i] = joint
    return skel.transpose(2,0,1)

In [53]:
def get_output_name(path):
    pstr = "_".join(path.split("/")[-1].split(".")[0].split("_")[:-1])
    return f"{pstr}_keypoints.csv"

In [60]:
def main():
    # List all the files in dir
    kp_path = "Movements/Kinect/Positions/"
    ka_path = "Movements/Kinect/Angles/"
    kinect_positions = sorted([kp_path+f for f in os.listdir(kp_path) if f.endswith('.txt')])
    kinect_angles = sorted([ka_path+f for f in os.listdir(ka_path) if f.endswith('.txt')])

    assert len(kinect_positions) == len(kinect_angles)
    
    N = len(kinect_positions)

    for i in tqdm(range(N)):
        # get a position/Angle file path 
        pos_path = kinect_positions[i]
        ang_path = kinect_angles[i]
        
        # get the output name
        op_name = get_output_name(pos_path)
        
        # create the directory you want to save to
        op_dir = "Movements/dataset_kp_kinect/"
        if not os.path.isdir(op_dir):
            os.makedirs(op_dir)

        # read the text files
        pos_data = genfromtxt(pos_path)
        ang_data = genfromtxt(ang_path)

        assert pos_data.shape[0] == ang_data.shape[0]
        
        num_frames = pos_data.shape[0]

        p_data = pos_data.T.reshape(num_kp, num_axes, -1)
        a_data = ang_data.T.reshape(num_kp, num_axes, -1)

        # tranform relative coordinates to absolute 
        skel = rel2abs(p_data, a_data, num_frames)

        x = skel.reshape(num_frames, -1)
        
        # save array as ".csv"
        np.savetxt(os.path.join(op_dir, op_name), x, delimiter=",")

In [61]:
main()

100%|██████████| 100/100 [01:47<00:00,  1.07s/it]
